<a href="https://colab.research.google.com/github/naseebjoon/bark/blob/main/hindi_long_form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Bark

In [9]:
pip install git+https://github.com/suno-ai/bark.git

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-7l03fhns
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-7l03fhns
  Resolved https://github.com/suno-ai/bark.git to commit f4f32d4cd480dfec1c245d258174bc9bde3c2148
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Environment Setup

In [10]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from IPython.display import Audio
import nltk  # we'll use this to split into sentences
import numpy as np

from bark.generation import (
    generate_text_semantic,
    preload_models,
)
from bark.api import semantic_to_waveform
from bark import generate_audio, SAMPLE_RATE
# Download the punkt resource
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Load Model

In [12]:
preload_models()

In [13]:
script = """
अरे, क्या तुमने इस नए टेक्स्ट-टू-ऑडियो मॉडल "Bark" के बारे में सुना है?
कहा जा रहा है कि यह अभी तक का सबसे यथार्थवादी और प्राकृतिक-साउंडिंग
टेक्स्ट-टू-ऑडियो मॉडल है। लोग कह रहे हैं कि यह बिलकुल किसी असली व्यक्ति
के बोलने जैसा लगता है। मुझे लगता है कि यह उन्नत मशीन लर्निंग एल्गोरिदम का
उपयोग करता है ताकि मानव आवाज़ की बारीकियों को विश्लेषित और समझा जा सके,
और फिर उन्हीं बारीकियों को अपनी आवाज़ में दोहराया जा सके। यह काफी
प्रभावशाली है, और मुझे लगता है कि इसे ऑडियोबुक्स या पॉडकास्ट जैसे
कामों में उपयोग किया जा सकता है। दरअसल, मैंने सुना है कि कुछ प्रकाशक
पहले ही Bark का उपयोग करके ऑडियोबुक्स बनाना शुरू कर चुके हैं। यह
बिल्कुल ऐसा होगा जैसे आपके पास खुद का एक निजी वॉइसओवर कलाकार हो।
मुझे सच में लगता है कि Bark टेक्स्ट-टू-ऑडियो तकनीक की दुनिया में
क्रांतिकारी बदलाव लाने वाला है।
""".replace("\n", " ").strip()

In [14]:
sentences = nltk.sent_tokenize(script)

In [ ]:
GEN_TEMP = 0.6
SPEAKER = "v2/hi_speaker_3"
silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence

pieces = []
for sentence in sentences:
    semantic_tokens = generate_text_semantic(
        sentence,
        history_prompt=SPEAKER,
        temp=GEN_TEMP,
        min_eos_p=0.05,  # this controls how likely the generation is to end
    )

    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)
    pieces += [audio_array, silence.copy()]

In [ ]:
Audio(np.concatenate(pieces), rate=SAMPLE_RATE)